# Building a Conversational AI Agent Using LangGraph
This guide is focused on using LangGraph to build a conversational AI agent. LangGraph is a powerful framework for constructing agentic systems—AI-driven systems that can handle tasks like decision-making, problem-solving, and interacting with users through natural language. LangGraph’s components are designed to be reusable and customizable, enabling developers to create intelligent agents with minimal effort.

## 1. Install dependencies
To install the necessary dependencies, run the following command:

In [ ]:
!pip install -qU langgraph "langchain[google-genai]"

LangGraph and LangChain are required to build and interact with AI agents. `langchain` helps us interact with language models like Gemini.

#2. Initialize Google Gemini Chat Model

In this step, we are initializing the Google Gemini chat model using LangChain’s init_chat_model function. The model is designed to interact with the user and generate responses.

In [ ]:
# Importing necessary libraries
import getpass  # Used to securely prompt the user for sensitive input like an API key
import os  # Used to interact with the operating system, like accessing environment variables

# Checking if the GOOGLE_API_KEY environment variable is set
if not os.environ.get("GOOGLE_API_KEY"):
    # If the API key is not set, prompt the user to enter it securely
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

# Importing the necessary function from LangChain to initialize the chat model
from langchain.chat_models import init_chat_model

# Initializing the Google Gemini chat model with specific settings
model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")


**Explanation:**

**getpass:** This library is used to securely prompt the user to input sensitive information, such as an API key. It doesn't display the input as it's typed, ensuring security.

**os:** The os library allows interaction with the operating system. Here, it's used to check if the API key is already set in the environment and to set the API key if not.

**os.environ.get("GOOGLE_API_KEY"):** This checks if the environment variable GOOGLE_API_KEY is already set. If it's not, the user is prompted to enter it.

**init_chat_model:** This function from the langchain.chat_models module is used to initialize the Google Gemini chat model (version 2.0, with the "flash" configuration) for generating responses.

## 3. Create an agent
Now we will create an agent using `create_react_agent` and attach a tool to fetch the weather for a city.

In [ ]:
from langgraph.prebuilt import create_react_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_react_agent(
    model="google_genai:gemini-2.0-flash",
    tools=[get_weather],
    prompt="You are a helpful assistant"
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)


### Explanation
- **create_react_agent**: A function from LangGraph that creates a conversational agent.
- **Model**: We use 'gemini-2.0-flash' model from Google Gemini AI to power the agent.
- **Tools**: The `get_weather` function acts as a tool that the agent can call.
- **Invoke**: The agent responds to the user input, asking about the weather in San Francisco (sf).

## 4. Configure an LLM (Language Model)
We will now configure the language model's parameters, such as temperature for controlling response randomness.

In [14]:

from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent

# Initialize the LLM with a specified temperature (0 for deterministic responses)
model = init_chat_model(
    "google_genai:gemini-2.0-flash",
    temperature=0
)

# Create the agent with the configured model and weather tool
agent = create_react_agent(
    model=model,
    tools=[get_weather],
)


### Explanation
- **init_chat_model**: Initializes the model with parameters like `temperature`, which controls the randomness of the model's responses.
- This setup makes the model's behavior more predictable.

## 4. Add a Custom Prompt
Now we will define a custom static prompt to guide the agent's responses.

In [ ]:

from langgraph.prebuilt import create_react_agent

# Static prompt (fixed text that doesn't change)
agent = create_react_agent(
    model="google_genai:gemini-2.0-flash",
    tools=[get_weather],
    prompt="Never answer questions about the weather."
)

# Try invoking the agent with a weather-related question
response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

print(response)  # Expect an answer telling the agent not to answer weather questions


### Explanation
- **Static Prompt**: We defined a fixed instruction telling the agent not to answer any weather-related questions. This guides the agent's behavior and response.

## 5. Add Memory (Multi-turn Conversations)
We will now add memory to the agent so that it can remember past conversations for multi-turn interactions.

In [ ]:

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

# Set up the memory to store conversation state
checkpointer = InMemorySaver()

# Create the agent with memory
agent = create_react_agent(
    model="google_genai:gemini-2.0-flash",
    tools=[get_weather],
    checkpointer=checkpointer
)

# Run agent for a conversation with a unique thread ID to remember the session
config = {"configurable": {"thread_id": "1"}}

# First conversation
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config
)

# Second conversation, using the same thread_id to keep the context
ny_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what about new york?"}]},
    config
)

print(sf_response)
print(ny_response)


### Explanation
- **Memory**: Using `InMemorySaver`, the agent is able to store the state of the conversation, making it capable of understanding and continuing multi-turn conversations.
- **Thread ID**: This is used to uniquely identify a conversation session. By reusing the same `thread_id`, the agent remembers prior exchanges.

## 6. Configure Structured Output
Finally, we will configure the agent to return structured responses using a Pydantic model.

In [ ]:
from pydantic import BaseModel
from langgraph.prebuilt import create_react_agent

class WeatherResponse(BaseModel):
    conditions: str

agent = create_react_agent(
    model="google_genai:gemini-2.0-flash",
    tools=[get_weather],
    response_format=WeatherResponse
)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

response["structured_response"]

### Explanation
- **Pydantic Model**: We define a `WeatherResponse` model to specify that the response from the agent should contain a `conditions` field.
- **Structured Response**: The agent now returns the weather conditions in a structured, predictable format.